### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
24,38894296,121339,79085,51531,0,0,974,2,2022-03-07 22:00,2022-03-07 22:00,0,2,1171,4,1
25,38894297,121339,79085,51531,0,0,974,3,2022-03-08 22:00,2022-03-08 22:00,0,2,1284,4,1
26,38893428,76933,79085,3,0,0,974,4,2022-03-09 19:13,2022-03-09 19:15,3,0,4557,3,1
27,38901346,10516,79085,51530,0,0,974,6,2022-03-11 23:00,2022-03-11 23:00,0,1,4591,4,1
28,38901347,10516,79085,51530,0,0,975,0,2022-03-13 15:00,2022-03-13 15:04,1,1,4344,3,1
29,38897091,122818,79085,1002,0,60,975,1,2022-03-14 21:29,2022-03-15 01:24,2,0,9247,5,1
30,38900874,121094,79085,3,0,0,975,4,2022-03-16 21:59,2022-03-16 22:00,0,0,5193,1,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

129

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,121339,39875,31825,3,8,4,2,1,Nada normal,7,7,7,38894296
1,79085,30195,38245,2,13,7,2,0,3-4-3a,7,7,7,38894296
0,121339,46827,44236,4,9,0,0,0,Nada normal,7,7,7,38894297
1,79085,46498,49089,1,11,0,0,0,5-2-3e,7,6,7,38894297
0,76933,21160,35880,5,13,2,0,0,AlberoGuerraCube,8,7,9,38893428
1,79085,48996,34276,0,1,2,2,0,5-3-2c,8,6,7,38893428
0,10516,27027,37023,1,9,0,0,0,elima,6,7,8,38901346
1,79085,42934,32938,1,4,2,0,0,5-3-2c,7,7,7,38901346
0,10516,30551,38022,0,6,1,0,0,DEF 42131MM,7,7,10,38901347
1,79085,39405,31934,0,2,1,0,0,5-2-3d,6,7,7,38901347


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

258

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2074

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,124619,0,0,0,0,0,0,0,12,2074
1,121353,0,0,0,0,0,0,0,11,2074
2,80171,0,0,0,0,0,0,0,10,2074
3,124282,0,0,0,0,0,0,0,9,2074
4,121268,0,0,0,0,0,0,0,8,2074
5,80374,0,0,0,0,0,0,0,7,2074
6,31199,0,0,0,0,0,0,0,6,2074
7,90335,0,0,0,0,0,0,0,5,2074
8,47712,0,0,0,0,0,0,0,4,2074
9,142661,0,0,0,0,0,0,0,3,2074


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

1

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()